##### Synthetic Paraphrased and Negated Caption Generation using 2 LLMs (Phi-4 & Mistral)

In [1]:
import pandas as pd
import ollama

In [ ]:
df = pd.read_csv('ccneg_paraphrased.csv') # Load csv file with original caption (e.g. CC-Neg dataset). The file should contain a column of filename and a column of caption. Actual file may vary by name.

In [3]:
def verify_with_mistral(original, paraphrased, false_sentence):
    """ Use Mistral to verify the generated sentences """
    
    # Verify paraphrased sentence (should be similar)
    prompt_verify_paraphrase = f"""
    The original sentence is: "{original}"
    The generated paraphrase is: "{paraphrased}"
    Does the paraphrase accurately retain the meaning of the original? Respond with "Yes" or "No".
    """
    response_paraphrase = ollama.chat(model="mistral", 
                                      messages=[{"role": "user", "content": prompt_verify_paraphrase}], 
                                      options={"temperature": 0.1})
    is_paraphrase_valid = "Yes" in response_paraphrase["message"]["content"]

    # Verify false sentence (should contradict)
    prompt_verify_false = f"""
    The original sentence is: "{original}"
    The generated false sentence is: "{false_sentence}"
    Does the false sentence contradict or misrepresent the original? Respond with "Yes" or "No".
    """
    response_false = ollama.chat(model="mistral", 
                                 messages=[{"role": "user", "content": prompt_verify_false}], 
                                 options={"temperature": 0.1})
    is_false_valid = "Yes" in response_false["message"]["content"]

    return is_paraphrase_valid, is_false_valid

In [4]:
def generate_and_verify(text):
    """Generate similar and false sentences with phi-4 and verify with mistral"""
    
    # Generate similar and false sentences using phi-4
    prompt_paraphrase = f"Rephrase the following sentence with the same meaning:\n{text}"
    response_paraphrase = ollama.chat(model="phi4", 
                                      messages=[{"role": "user", "content": prompt_paraphrase}], 
                                      options={"temperature": 0.1})
    paraphrased_sentence = response_paraphrase["message"]["content"]

    prompt_false = f"Generate a completely false version of this sentence:\n{text}"
    response_false = ollama.chat(model="phi4", 
                                 messages=[{"role": "user", "content": prompt_false}], 
                                 options={"temperature": 0.1})
    false_sentence = response_false["message"]["content"]

    # Verify the outputs with Mistral
    is_paraphrase_valid, is_false_valid = verify_with_mistral(text, paraphrased_sentence, false_sentence)

    # If mistral rejects, regenerate using mistral
    if not is_paraphrase_valid:
        response_paraphrase = ollama.chat(model="mistral", 
                                          messages=[{"role": "user", "content": prompt_paraphrase}], 
                                          options={"temperature": 0.1})
        paraphrased_sentence = response_paraphrase["message"]["content"]

    if not is_false_valid:
        response_false = ollama.chat(model="mistral", 
                                     messages=[{"role": "user", "content": prompt_false}], 
                                     options={"temperature": 0.1})
        false_sentence = response_false["message"]["content"]

    return paraphrased_sentence, false_sentence, is_paraphrase_valid, is_false_valid

In [ ]:
# Apply the function to generate and verify new sentences
df[["paraphrased", "negation", "cap_Mistral_Verified_Similar", "cap_Mistral_Verified_False"]] = \
    df["caption"].apply(lambda x: pd.Series(generate_and_verify(x)))

In [ ]:
df.to_csv('ccneg_paraphrased.csv', index=False) # This is the updated version with new caption columns and verification from Mistral. This is the file for SemCLIP training.